In [2]:
# Import dependencies for scraping: pandas, beautiful soup, requests/splinter
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\linds\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [11]:
# Nasa Mars News
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
# URL info
nasa_mars_url = "https://mars.nasa.gov/news"
browser.visit(nasa_mars_url)
html = browser.html
soup = bs(html,'html.parser')

# Grab latest news title and paragraph text. Assign the text to variables that you can reference later.
# Title: after inspecting url found news title with "content_title" class 
news_title = soup.find_all('div', class_='content_title')[1].text

# Paragraph: after inspecting url found summary paragraph on website within article_teaser_body

news_p = soup.find_all('div', class_='article_teaser_body')[0].text

# Print
print("Most recent title from mars.nasa.gov/news:")
print("------------------------------------------")
print(news_title)
print("------------------------------------------")
print("Corresponding first paragraph:")
print("------------------------------------------")
print(news_p)

Most recent title from mars.nasa.gov/news:
------------------------------------------
Testing Proves Its Worth With Successful Mars Parachute Deployment
------------------------------------------
Corresponding first paragraph:
------------------------------------------
The giant canopy that helped land Perseverance on Mars was tested here on Earth at NASA’s Wallops Flight Facility in Virginia.


In [14]:
# JPL Mars Space Images - Featured Image
# Visit the url for JPL Featured Space Image here https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html

# first go to the website
jpl_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(jpl_url)
html = browser.html
soup = bs(html,'html.parser')

# grab the featured image by going through url, div class "floating text area," href a tag has featured image link
featured_image = soup.find('div',class_='floating_text_area').a['href']

# url for featured image
featured_image_url = jpl_url + featured_image
featured_image_url


'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.htmlimage/featured/mars3.jpg'

In [14]:
# Mars Facts
# Visit the Mars Facts webpage and use Pandas to scrape the table containing facts about the planet including Diameter, 
# Mass, etc.

mars_facts_url = "https://space-facts.com/mars/"
browser.visit(mars_facts_url)

mars_table = pd.read_html(mars_facts_url)
mars_facts = pd.DataFrame(mars_table)

#Use Pandas to convert the data to a HTML table string.
mars_facts_html = mars_facts.to_html()
mars_facts_html.replace('\n','')
print(mars_facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos &amp; Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Mars - Earth Comparison             Mars            Earth
0               Diameter:         6,779 km        12,742 km
1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
2                  Moons:                2              

In [28]:
# Mars Hemispheres
# Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres.

# first go to the website
mars_hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hemisphere_url)
html = browser.html
soup = bs(html,'html.parser')

# grab each hemisphere image by going through url, div class "collapsible results" highlights all hemispheres (items)
mars_hemispheres = soup.find('div',class_='collapsible results')
mars_item = mars_hemispheres.find_all('div',class_='item')

# initialize hemisphere image url list
hemisphere_image_urls = []

# loop through each item to get title and image url
for item in mars_item:
    # error handling
    try:
        # grab title in h3 text of div class "description"
        hemisphere = item.find('div', class_='description')
        title = hemisphere.h3.text
        
        # grab image url in <a href a\>
        # go to hemisphere url
        hemisphere_url = hemisphere.a['href']
        browser.visit('https://astrogeology.usgs.gov' + hemisphere_url)
        html = browser.html
        soup = bs(html,'html.parser')
        image_url = soup.find('li').a['href']
        
        if (title and image_url):
            # print results
            print('-------------------------------------')
            print(title)
            #print(image_url)
            
        # create dictionary for title and url
        hemisphere_dict={
            'title': title,
            'image_url': image_url
        }
        hemisphere_image_urls.append(hemisphere_dict)
    except Exception as e:
        print(e)

hemisphere_image_urls


-------------------------------------
Cerberus Hemisphere Enhanced
-------------------------------------
Schiaparelli Hemisphere Enhanced
-------------------------------------
Syrtis Major Hemisphere Enhanced
-------------------------------------
Valles Marineris Hemisphere Enhanced


[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]